In [1]:
import sys

In [2]:
sys.path.append('../..')

In [15]:
from datetime import datetime

In [3]:
from app import db
from models import Type as DBType

In [6]:
d = """
reference-entry	Entries within reference materials, such as encyclopedias and dictionaries.
standard	Standards developed by Standards Development Organizations (SDOs) or consortia.
dataset	Dataset records capture information about one or more database records or collections.
erratum	Corrections of existing works.
paratext	Paratext are works that represent stuff that is *about* the venue (such as a journal)—rather than a scholarly work properly speaking. These include things like front-covers, back-covers, tables of contents, and the journal itself
article	Research works that generally contains original research or reviews existing research. This includes journal articles, conference articles, and preprint articles.
editorial	Generally short works written by the editors of journals or other publications.
grant	Works associated with research grants, which provide funding to support research.
peer-review	Reviews, reports, or comments about another work.
report	Technical reports and working papers.
letter	These are typically letters to the editors at journals, though it may include other types of letters.
book-chapter	A single chapter within a book. It is common to publish original research as book chapters in certain fields, such as computer science.
dissertation	Dissertations and theses are works representing documents submitted by someone in completion of an academic degree or professional qualification.
other	Works from Crossref marked with type "other".
book	Scholarly books including reference books and monographs.
"""
d = [line.split('\t') for line in d.splitlines() if line]

In [17]:
now = datetime.utcnow().isoformat()
for type_id, description in d:
    t = db.session.query(DBType).filter_by(type_id=type_id).one()
    t.description = description
    t.updated_date = now
    db.session.add(t)
db.session.commit()